In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2 as cv
from pathlib import Path
import tqdm
import warnings
import shutil

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

# Master Category Classification #

In [2]:
#define paths and read data
root = 'C:/Users/hp/Desktop/S8/Data science/'
style_file = 'styles.csv'
image_folder = root + '/images/'

In [3]:

print(root+style_file)
styles = pd.read_csv(Path(style_file),error_bad_lines=False)

C:/Users/hp/Desktop/S8/Data science/styles.csv


b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [4]:
styles.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [5]:
sub_bool = [styles['masterCategory'].value_counts()<=30][0]
sub = styles['masterCategory'].value_counts()
sub_restant = [] #les sous catégories sous représentées que l'on va regrouper sous la classe 'Others'
sub_classified = [] # les sous catégories que l'on va classifier
#diviser les catégories
for i in range(len(sub_bool)):
    if sub_bool[i]:
        sub_restant.append(sub.index[i])
    else :
        sub_classified.append(sub.index[i])

sub_classified.append('Others')


In [7]:

target = 'masterCategory'

categories = sub_restant

df_train = styles[['id',target]]

df_train[target] = df_train[target].replace(categories, 'Others')


df_train['class'] = pd.factorize(df_train[target])[0] #pd.factorize()=codes, uniques
print("Data Shape: ", str(df_train.shape))
print(df_train[target].value_counts())

Data Shape:  (44424, 3)
Apparel          21397
Accessories      11274
Footwear          9219
Personal Care     2403
Free Items         105
Others              26
Name: masterCategory, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['id'],df_train['masterCategory'],test_size=0.2,stratify=df_train['class'],random_state=42)
print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

Training data and target sizes: 
(35539,), (35539,)
Test data and target sizes: 
(8885,), (8885,)


#### Création de dossier par catégorie et divisé selon train et test ####
Cela sert à implémenter les données dans pytorch qui préfère lire les données d'entrainement et de test lorsqu'elles sont placées de cette manière.

In [12]:
for sub in sub_restant:
    os.makedirs('./data_cloth/train/'+str(sub))
    os.makedirs('./data_cloth/test/'+str(sub))
    
for i in X_train.index:
    continue
    shutil.copy('C:/Users/hp/Desktop/S8/Data science/images/'+str(X_train[i])+'.jpg', './data_cloth/train/'+str(y_train[i]))
for i in X_test.index:
    continue
    shutil.copy('C:/Users/hp/Desktop/S8/Data science/images/'+str(X_test[i])+'.jpg', './data_cloth/test/'+str(y_test[i]))

#### Preprocessing des données : scaling, normalisation, convertion en tensor .. ####
Aussi, ici on charge nos données à l'aide du dataloader : **il permettra d'itérer sur nos données dans le forward et backward pass de notre réseau de neurones**. **Le batchsize représente le nombre de sample sur chaque forward et backward pass du réseau de neurone**. Ainsi, ici, on aura à chaque itération d'entrainement de notre réseau de neurones 4 samples qui passeront dans le réseau et les poids des liaisons du neurones seront mis à jour avec la chain rule ect..  grâce à ces 4 samples avec la méthode puis on itère sur les 4 samples suivants ect ...

In [13]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

#data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean, std)]),'val': transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean, std)]),}


#transformation sur les images
data_transforms = {
    'train': transforms.Compose([
        transforms.Normalize(mean, std),
        transforms.ToTensor()#Convert a PIL Image or numpy.ndarray to tensor.
    ]),
    'test': transforms.Compose([
        transforms.Normalize(mean, std),
        transforms.ToTensor()
    ]),
}

data_dir = './data_cloth'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ### cette commande permet de travailler sur le GPU si on en a un, sur le CPU sinon.
print(class_names)

FileNotFoundError: Found no valid file for the classes Home, Sporting Goods. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:

#mean = np.array([0.5, 0.5, 0.5])
#std = np.array([0.25, 0.25, 0.25])
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

print(class_names[classes.numpy()[0]])

Ici, on définit la fonction permettant d'entrainer notre modèle: **le num_epochs représente le nombre de fois où l'ensemble des données effectue un forward et backward pass**, on choisit notre fonction de perte en argument, notre algorithme d'optimisation et un scheduler un peu gadget qui permet d'ajuster le learning rate en fonction du nombre d'epochs (globalement cela permet d'affiner l'optimisation et d'avoir de meilleures performances) 

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode
                n_class_correct = [0 for i in range(len(class_names))]
                n_class_samples = [0 for i in range(len(class_names))]
            
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) #retourne valeur et indice 
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                if phase =='test':
                    for i in range(4):
                        
                        label = labels[i]
                        pred = preds[i]
                        if (label == pred):
                            n_class_correct[label] += 1 
                        n_class_samples[label]+=1
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'test':
                for i in range(len(class_names)):
                    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
                    print(f'accuracy of {class_names[i]} is {acc:.4f}')


            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model




Ici, chaque classe correspond à un réseau de neurones, on code chaque couches à la main : on choisit le nombre neurones en entrée et sortie, de plus on choisit nos fonctions d'activations.

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes ):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, hidden_size2)
        self.l3 = nn.Linear(hidden_size2, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

class MLP2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes ):
        super(MLP2, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out  

### Network freezed (except final layer) ###
Ici, on fait du transfert learning, c'est à dire qu'on utilise un réseau de neurone Resnet déjà entrainé sur une banque d'image (d'où l'argument pretrained = True) donc les poids sont déjà initialisé : on 'colle' ensuite au bout de ce Resnet un MLP que l'on code nous même. Ici on choisit de 'freeze' les poids du resnet : ainsi, seuls sont entrainés les poids de notre MLP 

In [ ]:
#### ConvNet as fixed feature extractor ####
# Here, we need to freeze all the network except the final layer.
# We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
hidden_size2 = 16
hidden_size1 = 64

model_conv.fc = MLP(num_ftrs, hidden_size1, hidden_size2,len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.005, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.2)

model_conv = train_model(model_czonv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=10)

### The whole network is trained here ###
Ici, on entraîne les poids de tout le réseau de neurone i.e Resnet + notre MLP : logiquement les résultats sont bien meilleurs, mais c'est aussi beaucoup plus long

In [ ]:
#### Finetuning the convnet ####
# Load a pretrained model and reset final fully connected layer.

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features


hidden_size = 16
hidden_size1 = 64

model.fc = MLP2(num_ftrs, hidden_size, len(class_names))



model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.005)


step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=15)

# subCategory Classification #
#### Ici, on fait la même chose qu'au dessus mais sur les subCategory, les résultats seront donc un peu moins précis car on rentre plus dans le détail. ####

In [ ]:
sub_bool = [styles['subCategory'].value_counts()<=30][0]
sub = styles['subCategory'].value_counts()
sub_restant = [] #les sous catégories sous représentées que l'on va regrouper sous la classe 'Others'
sub_classified = [] # les sous catégories que l'on va classifier

for i in range(len(sub_bool)):
    if sub_bool[i]:
        sub_restant.append(sub.index[i])
    else :
        sub_classified.append(sub.index[i])

sub_classified.append('Others')

In [ ]:

target = 'subCategory'

categories = sub_restant

df_train = styles[['id',target]]

df_train[target] = df_train[target].replace(categories, 'Others')


df_train['class'] = pd.factorize(df_train[target])[0]
print("Data Shape: ", str(df_train.shape))
print(df_train[target].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['id'],df_train['subCategory'],test_size=0.2,stratify=df_train['class'],random_state=42)
print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

In [ ]:
for sub in sub_classified:
    os.makedirs('./data_clothes_sub/train/'+str(sub))
    os.makedirs('./data_clothes_sub/test/'+str(sub))
    
for i in X_train.index:
    if os.path.isfile('./images/'+str(X_train[i])+'.jpg'):
        shutil.copy('./images/'+str(X_train[i])+'.jpg', './data_clothes_sub/train/'+str(y_train[i]))
for i in X_test.index:
    if os.path.isfile('./images/'+str(X_test[i])+'.jpg'):
        shutil.copy('./images/'+str(X_test[i])+'.jpg', './data_clothes_sub/test/'+str(y_test[i]))

In [ ]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

#data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean, std)]),'val': transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean, std)]),}


data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((60,80)),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Scale((60,80)),
        transforms.ToTensor()
    ]),
}

data_dir = './data_clothes_sub'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

In [ ]:
#### Finetuning the convnet ####
# Load a pretrained model and reset final fully connected layer.

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

hidden_size = 16
hidden_size1 = 64

model.fc = MLP2(num_ftrs, hidden_size, len(class_names))



model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.005)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=12)

# Article Type Classification #

#### Ici, on rentre encore plus dans le détail avec la classification du type d'articles ####

In [ ]:
article_bool = [styles['articleType'].value_counts()<=40][0]
article = styles['articleType'].value_counts()
article_restant = []
article_classified = []

for i in range(len(article_bool)):
    if article_bool[i]:
        article_restant.append(article.index[i])
    else:
        article_classified.append(article.index[i])

article_classified.append('Others')

In [ ]:

target = 'articleType'

categories = article_restant

df_train = styles[[target,'id']]

df_train[target] = df_train[target].replace(categories, 'Others')

df_train['class'] = pd.factorize(df_train[target])[0]
print("Data Shape: ", str(df_train.shape))
print(df_train[target].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['id'],df_train['articleType'],test_size=0.2,stratify=df_train['class'],random_state=42)
print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

In [ ]:
for article in article_classified:
    os.makedirs('./data_clothes_type/train/'+str(article))
    os.makedirs('./data_clothes_type/test/'+str(article))
    
for i in X_train.index:
    if os.path.isfile('./images/'+str(X_train[i])+'.jpg'):
        shutil.copy('./images/'+str(X_train[i])+'.jpg', './data_clothes_type/train/'+str(y_train[i]))
for i in X_test.index:
    if os.path.isfile('./images/'+str(X_test[i])+'.jpg'):
        shutil.copy('./images/'+str(X_test[i])+'.jpg', './data_clothes_type/test/'+str(y_test[i]))

In [ ]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

#data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean, std)]),'val': transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean, std)]),}


data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((60,80)),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Scale((60,80)),
        transforms.ToTensor()
    ]),
}

data_dir = './data_clothes_type'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

hidden_size2 = 16
hidden_size1 = 64

model.fc = MLP(num_ftrs, hidden_size1, hidden_size2, len(class_names))



model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.005)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=12)